# Notes

Used simplified version of the data used in the paper "Embeddings from protein language models predict conservation and variant effects" by Marquet et al.

**Data explanation**:
- `seq_and_conservation.txt`: FASTA file with 3 lines per entry: amino acid sequence, cotinuous conservation scores (ignored), conservation classes between 1 and 9 (1 = very variable, 9 = very conserved)
- `train_ids.txt`: IDs of the proteins for training (9392 proteins)
- `val_ids.txt`: IDs of the proteins for validation (555 proteins)
- `test_ids.txt`: IDs of the proteins for testing (519 proteins)

**Proposed splits**:
- `sampled`: Randomly split sequences into `train`/`test` with 95/5% probability.

This is a well-known dataset used to validate the behavior of code and models. Only provided a `sampled` split for this purpose.

# Configs & Imports

In [1]:
from pathlib import Path

from pandas import DataFrame, read_json
import json

from Bio import SeqIO

%load_ext autoreload
%autoreload 2

In [2]:
# Where the raw data is stored and where processed data will be deposited
data_path = Path('') / '..' / 'data' / 'conservation'

sequences = data_path / 'seq_and_conservation.txt'
train = data_path / 'train_ids.txt'
validation = data_path / 'val_ids.txt'
test = data_path / 'test_ids.txt'

split_path = Path('') / '..' / 'splits' / 'cons'

# Obtain original dataset

In [3]:
parser = SeqIO.parse(sequences, "fasta")

seqs_and_cons = list(parser)

dataset = DataFrame(columns = ["id", "sequence", "conservations"])
for i in range(0, len(seqs_and_cons), 3):
    id = seqs_and_cons[i].description[1:].replace(' ', '-')
    sequence = seqs_and_cons[i].seq
    conservations = seqs_and_cons[i+2].seq
    
    dataset = dataset.append({"id": id,
                              "sequence": str(sequence), 
                              "conservations": str(conservations)}, ignore_index=True)

In [4]:
dataset

,id,sequence,conservations
0,3p6z-C,AHHHHHHVGTWENLYFQSIPDDDEDSYEIFEPPESTVMATRKMHDR...,"6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,5,7,7,3,4,7,3,..."
1,6fb3-A,TGSLVSLIRGQVVTTDGTPLVGVNVSFVKYPKYGYTITRQDGMFDL...,"6,4,9,8,6,7,9,6,9,9,7,9,3,8,1,7,7,6,8,8,6,9,8,..."
2,2oqg-A,QGMTVGTYAELASVFAALSDETRWEILTELGRADQSASSLATRLPV...,"1,6,6,7,5,5,4,1,6,3,4,6,2,6,8,5,9,8,7,9,5,8,9,..."
3,3ix7-A,SNAPRGGKVLDTSVLVDGRVAEVAAVGFLEGPLWVPHFVLKELQHF...,"6,4,1,1,1,1,1,8,5,8,9,9,9,7,8,9,9,9,9,8,3,7,6,..."
4,5y9q-A,GTISIGCSSLIGQTLLPEVLSLYNAQFPNVEIQVQVGSTEQIKANH...,"9,8,9,7,7,8,7,9,8,7,6,8,7,8,5,9,9,1,6,9,6,1,7,..."
...,...,...,...
10502,1ws8-A,MATVHKVGDSTGWTTLVPYDYAKWASSNKFHVGDSLLFNYNNKFHN...,"8,9,6,3,6,4,9,9,6,2,1,7,9,3,2,4,3,2,3,5,7,3,3,..."
10503,4qa8-A,GAMDPEFGKKPTTASSPSPGSPSPEAQQILQDSSKATKGLHSVHVV...,"7,6,4,4,6,4,1,5,1,1,1,1,3,3,3,3,1,1,2,1,1,1,6,..."
10504,1v2z-A,STAFFFRRMSPADKRKLLDELRSIYRTIVLEYFNTDAKVNERIDEF...,"6,7,1,1,6,2,6,3,7,1,1,1,4,3,1,1,1,1,1,1,7,4,1,..."
10505,3h7h-B,GSHMDPNLWTVKCKIGEERATAISLMRKFIAYQFTDTPLQIKSVVA...,"6,9,1,3,9,8,6,8,8,5,7,7,8,5,2,9,6,9,6,2,6,6,2,..."


# Splits

## sampled

In [5]:
# Obtain IDs of Train, Validation and Test proteins
with open(train) as train_file:
    train_ids = [line.rstrip() for line in train_file]

with open(validation) as validation_file:
    validation_ids = [line.rstrip() for line in validation_file]
    
with open(test) as test_file:
    test_ids = [line.rstrip() for line in test_file]

In [6]:
# Let's create the split dataset
sampled = DataFrame(columns = ["sequence", "target", "set", "validation"])

# Add train samples
tmp = dataset[dataset["id"].isin(train_ids)][["sequence", "conservations"]]
tmp.rename(columns = {"conservations": "target"}, inplace = True)
tmp.insert(2, "set", ["train"]*len(tmp))
sampled = sampled.append(tmp)

# Add validation samples
tmp = dataset[dataset["id"].isin(validation_ids)][["sequence", "conservations"]]
tmp.rename(columns = {"conservations": "target"}, inplace = True)
tmp.insert(2, "set", ["train"]*len(tmp))
tmp.insert(2, "validation", [True]*len(tmp))
sampled = sampled.append(tmp)

# Add test samples
tmp = dataset[dataset["id"].isin(test_ids)][["sequence", "conservations"]]
tmp.rename(columns = {"conservations": "target"}, inplace = True)
tmp.insert(2, "set", ["test"]*len(tmp))
sampled = sampled.append(tmp)

sampled = sampled.reset_index(drop = True)

In [7]:
sampled

,sequence,target,set,validation
0,AHHHHHHVGTWENLYFQSIPDDDEDSYEIFEPPESTVMATRKMHDR...,"6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,6,5,7,7,3,4,7,3,...",train,NaN
1,TGSLVSLIRGQVVTTDGTPLVGVNVSFVKYPKYGYTITRQDGMFDL...,"6,4,9,8,6,7,9,6,9,9,7,9,3,8,1,7,7,6,8,8,6,9,8,...",train,NaN
2,QGMTVGTYAELASVFAALSDETRWEILTELGRADQSASSLATRLPV...,"1,6,6,7,5,5,4,1,6,3,4,6,2,6,8,5,9,8,7,9,5,8,9,...",train,NaN
3,SNAPRGGKVLDTSVLVDGRVAEVAAVGFLEGPLWVPHFVLKELQHF...,"6,4,1,1,1,1,1,8,5,8,9,9,9,7,8,9,9,9,9,8,3,7,6,...",train,NaN
4,GTISIGCSSLIGQTLLPEVLSLYNAQFPNVEIQVQVGSTEQIKANH...,"9,8,9,7,7,8,7,9,8,7,6,8,7,8,5,9,9,1,6,9,6,1,7,...",train,NaN
...,...,...,...,...
10461,MTGSNSHITILTLNINGLNSAIKRHRLASWIKSQDPSVCCIQETHL...,"6,5,5,2,6,5,4,5,6,7,4,8,6,9,7,8,8,6,7,6,4,3,8,...",test,NaN
10462,GAMATTKIRIFVPATNSPELRWELTLFALDVIRSPSAAESMKVGAA...,"8,6,4,7,7,4,5,7,4,8,5,6,6,5,6,2,3,4,1,1,9,4,1,...",test,NaN
10463,KFSAKRLPSTRLGTFLENRVNDFLRRQNHPESGEVTVRVVHASDKT...,"6,3,1,7,6,3,8,5,1,6,2,8,7,1,1,6,9,1,9,8,8,1,1,...",test,NaN
10464,GAKNYYDITLALAGICQSARLVQQLAHQGHCDADALHVSLNSIIDM...,"6,8,8,6,1,2,5,8,7,6,9,9,9,7,6,7,9,8,7,1,7,9,6,...",test,NaN


In [8]:
sampled.to_csv(split_path / 'splits' / 'sampled.csv', index = False)